In [1]:
!pip install -q -U transformers peft trl bitsandbytes accelerate datasets sentence-transformers scikit-learn tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.2/494.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 42.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.31.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-adk 1.21.0 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
dopa

In [2]:
import json
from datasets import Dataset
from transformers import AutoTokenizer

# نام فایل نهایی شما که در مرحله قبل تولید کردیم
TRAIN_FILE = '/kaggle/input/datasets/daniyalghoreyshi/train-data/train_data_ready_for_finetuning.jsonl'
MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

print("در حال بارگذاری Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

def format_to_chat_template(example):
    """
    تبدیل داده‌های ما به قالب استاندارد چت برای آموزش Qwen
    """
    messages = [
        {
            "role": "system", 
            "content": "You are an expert AI specialized in Abductive Event Reasoning. First, briefly reason based on the provided Context. Then, output the uppercase letter(s) of the correct option(s) inside <ANSWER> tags."
        },
        {
            "role": "user", 
            "content": f"Context: {example['context']}\n\nTarget Event: {example['target_event']}\n\nOptions:\nA) {example['options']['A']}\nB) {example['options']['B']}\nC) {example['options']['C']}\nD) {example['options']['D']}\n\nReason and output final answer in <ANSWER> tags."
        },
        {
            "role": "assistant", 
            # این همان چیزی است که مدل باید یاد بگیرد تولید کند!
            "content": f"{example['reasoning']}\n<ANSWER>{example['golden_answer']}</ANSWER>"
        }
    ]
    # تبدیل لیست پیام‌ها به یک رشته متنی استاندارد با فرمت چت
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    return {"text": text}

# خواندن فایل و تبدیل به فرمت HuggingFace Dataset
print("در حال بارگذاری و قالب‌بندی دیتاست...")
data_list = []
with open(TRAIN_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line))

hf_dataset = Dataset.from_list(data_list)
# اعمال تابع قالب‌بندی روی کل دیتاست
formatted_dataset = hf_dataset.map(format_to_chat_template)

print(f"\n✅ دیتاست آماده شد! تعداد نمونه‌های آموزشی: {len(formatted_dataset)}")
print("-" * 50)
print("نمونه‌ای از داده آموزشی نهایی که به خورد مدل می‌رود:\n")
print(formatted_dataset[0]['text'][:500] + "...\n[ادامه دارد]")

در حال بارگذاری Tokenizer...


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

در حال بارگذاری و قالب‌بندی دیتاست...


Map:   0%|          | 0/1461 [00:00<?, ? examples/s]


✅ دیتاست آماده شد! تعداد نمونه‌های آموزشی: 1461
--------------------------------------------------
نمونه‌ای از داده آموزشی نهایی که به خورد مدل می‌رود:

<|im_start|>system
You are an expert AI specialized in Abductive Event Reasoning. First, briefly reason based on the provided Context. Then, output the uppercase letter(s) of the correct option(s) inside <ANSWER> tags.<|im_end|>
<|im_start|>user
Context: Two Iranian Drones Shot Down in Iraq by International Coalition: Officials Two drones launched from Iran towards Israel were shot down over Iraq by the US-led international coalition to defeat the Islamic State group, two Iraqi military official...
[ادامه دارد]


In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainerCallback
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

# ==========================================
# ۰. تعریف سیستم لاگ‌گیر اختصاصی برای Kaggle Commit
# ==========================================
class KaggleLogCallback(TrainerCallback):
    """
    این کلاس باعث می‌شود اطلاعات آموزش به صورت خط به خط و تمیز 
    در لاگ‌های پس‌زمینه (Commit) چاپ شوند.
    """
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            step = state.global_step
            epoch = logs.get("epoch", 0)
            loss = logs.get("loss", "N/A")
            lr = logs.get("learning_rate", "N/A")
            print(f"📊 Step: {step:04d} | Epoch: {epoch:.2f} | Loss: {loss:.4f} | LR: {lr:.2e}")

# ==========================================
print("در حال بارگذاری مدل پایه (با رفع اخطار dtype)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# ۱. بارگذاری مدل
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, 
    quantization_config=bnb_config, 
    device_map="auto",
    dtype=torch.float16
)

model.config.torch_dtype = torch.float16
model = prepare_model_for_kbit_training(model)

# ۲. تنظیمات LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# ۳. تنظیمات آموزش 
training_args = SFTConfig(
    output_dir="./qwen-abductive-adapter",
    per_device_train_batch_size=2,      
    gradient_accumulation_steps=4,      
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,                   # هر ۱۰ قدم لاگ چاپ می‌شود
    num_train_epochs=2,                 
    max_grad_norm=0.3,
    warmup_steps=15,                    
    fp16=True,                          
    bf16=False,                         
    report_to="none",
    dataset_text_field="text",          
    max_length=1500,
    disable_tqdm=True                   # 👈 خاموش کردن tqdm برای تمیز ماندن لاگ
)

# ۴. ساخت Trainer و اضافه کردن Callback اختصاصی
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    processing_class=tokenizer,         
    args=training_args,
    callbacks=[KaggleLogCallback()]     # 👈 فعال‌سازی لاگ‌گیر Kaggle
)

# ۵. پاکسازی قطعی BFloat16 
for name, param in trainer.model.named_parameters():
    if param.requires_grad:
        param.data = param.data.to(torch.float32)
    elif param.dtype == torch.bfloat16:
        param.data = param.data.to(torch.float16)

print("\n🚀 در حال شروع عملیات Fine-Tuning... (وضعیت هر 10 قدم گزارش می‌شود)")
trainer.train()

# ۶. ذخیره آداپتور آموزش دیده
ADAPTER_PATH = "./qwen-abductive-final-adapter"
trainer.model.save_pretrained(ADAPTER_PATH)
tokenizer.save_pretrained(ADAPTER_PATH)
print(f"\n🎉 آموزش تمام شد! وزن‌های جدید در پوشه {ADAPTER_PATH} ذخیره شدند.")

در حال بارگذاری مدل پایه (با رفع اخطار dtype)...


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

...

Adding EOS to train dataset:   0%|          | 0/1461 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1461 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1461 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.



🚀 در حال شروع عملیات Fine-Tuning... (وضعیت هر 10 قدم گزارش می‌شود)
📊 Step: 0010 | Epoch: 0.05 | Loss: 2.3655 | LR: 1.20e-04
{'loss': '2.366', 'grad_norm': '0.184', 'learning_rate': '0.00012', 'entropy': '1.512', 'num_tokens': '4.966e+04', 'mean_token_accuracy': '0.5553', 'epoch': '0.05472'}
📊 Step: 0020 | Epoch: 0.11 | Loss: 1.6483 | LR: 2.00e-04
{'loss': '1.648', 'grad_norm': '0.08482', 'learning_rate': '0.0001999', 'entropy': '1.731', 'num_tokens': '1.004e+05', 'mean_token_accuracy': '0.6342', 'epoch': '0.1094'}
📊 Step: 0030 | Epoch: 0.16 | Loss: 1.4367 | LR: 1.99e-04
{'loss': '1.437', 'grad_norm': '0.08906', 'learning_rate': '0.0001992', 'entropy': '1.467', 'num_tokens': '1.486e+05', 'mean_token_accuracy': '0.6728', 'epoch': '0.1642'}
📊 Step: 0040 | Epoch: 0.22 | Loss: 1.2960 | LR: 1.98e-04
{'loss': '1.296', 'grad_norm': '0.1145', 'learning_rate': '0.0001977', 'entropy': '1.319', 'num_tokens': '1.968e+05', 'mean_token_accuracy': '0.6924', 'epoch': '0.2189'}
📊 Step: 0050 | Epoch: 0.

In [4]:
import json
import re
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# ==========================================
# ۱. بارگذاری مدل فاین‌تیون شده برای تست
# ==========================================
print("در حال بارگذاری مدل برای Inference...")
base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    device_map="auto",
    torch_dtype=torch.float16
)
# چسباندن مغز آموزش دیده (آداپتور) به مدل پایه
model = PeftModel.from_pretrained(base_model, "./qwen-abductive-final-adapter")
tokenizer = AutoTokenizer.from_pretrained("./qwen-abductive-final-adapter")

# ==========================================
# ۲. توابع RAG و پردازش فایل تست
# ==========================================
embedder = SentenceTransformer('BAAI/bge-large-en-v1.5')

def clean_text(text):
    if not text or not isinstance(text, str): return ""
    return re.sub(r'\s+', ' ', text).strip()

def load_eval_data(questions_path, docs_path):
    # این تابع دقیقاً مشابه قبل است، با این تفاوت که روی داده‌های dev/test کار می‌کند
    with open(docs_path, 'r', encoding='utf-8') as f:
        docs_raw = json.load(f)
    
    docs_dict = {}
    if isinstance(docs_raw, list):
        for item in docs_raw:
            if not isinstance(item, dict): continue
            t_id = item.get("topic_id")
            if not t_id: continue
            if t_id not in docs_dict: docs_dict[t_id] = []
            if "docs" in item or "documents" in item:
                inner_docs = item.get("docs", item.get("documents", []))
                for d in inner_docs:
                    if isinstance(d, dict): docs_dict[t_id].append(d.get("text", d.get("content", d.get("doc", ""))))
                    elif isinstance(d, str): docs_dict[t_id].append(d)
            else:
                text_val = item.get("text", item.get("content", item.get("doc", "")))
                if text_val: docs_dict[t_id].append(text_val)

    merged = []
    with open(questions_path, 'r', encoding='utf-8') as f:
        for line in f:
            if not line.strip(): continue
            q_item = json.loads(line.strip())
            t_id = q_item.get("topic_id")
            related_docs = docs_dict.get(t_id, [])
            merged.append({
                "id": q_item.get("id"),
                "target_event": clean_text(q_item.get("target_event")),
                "options": {
                    "A": clean_text(q_item.get("option_A")),
                    "B": clean_text(q_item.get("option_B")),
                    "C": clean_text(q_item.get("option_C")),
                    "D": clean_text(q_item.get("option_D"))
                },
                "context": " ".join([clean_text(doc) for doc in related_docs if doc]),
                # در test_data واقعی این فیلد وجود ندارد، لذا اگر نبود None می‌گذاریم
                "golden_answer": q_item.get("golden_answer", None) 
            })
    return merged

def retrieve_relevant_context(sample, top_k=6):
    sentences = re.split(r'(?<=[.!?])\s+', sample['context'])
    sentences = [s.strip() for s in sentences if len(s.strip()) > 20]
    if not sentences: return ""
    query = sample['target_event'] + " " + " ".join(sample['options'].values())
    sentence_embeddings = embedder.encode(sentences)
    query_embedding = embedder.encode([query])
    similarities = cosine_similarity(query_embedding, sentence_embeddings).flatten()
    top_indices = similarities.argsort()[-top_k:]
    return " ".join([sentences[i] for i in sorted(top_indices)])

# ==========================================
# ۳. اجرای پیش‌بینی و ذخیره فرمت Codabench
# ==========================================
# برای تست نهایی، مسیر فایل test_data را اینجا قرار دهید
EVAL_QUESTIONS = 'semeval2026-task12-dataset/dev_data/questions.jsonl'
EVAL_DOCS = 'semeval2026-task12-dataset/dev_data/docs.json'
PREDICTIONS_OUTPUT = 'predictions.jsonl'

print("در حال بارگذاری داده‌های آزمون...")
eval_dataset = load_eval_data(EVAL_QUESTIONS, EVAL_DOCS)

print(f"شروع پیش‌بینی روی {len(eval_dataset)} نمونه...")
with open(PREDICTIONS_OUTPUT, 'w', encoding='utf-8') as out_file:
    for sample in tqdm(eval_dataset):
        
        # استخراج کانتکست با RAG
        smart_context = retrieve_relevant_context(sample, top_k=6)
        
        # ساخت پرامپت (بدون جواب)
        messages = [
            {"role": "system", "content": "You are an expert AI specialized in Abductive Event Reasoning. First, briefly reason based on the provided Context. Then, output the uppercase letter(s) of the correct option(s) inside <ANSWER> tags."},
            {"role": "user", "content": f"Context: {smart_context}\n\nTarget Event: {sample['target_event']}\n\nOptions:\nA) {sample['options']['A']}\nB) {sample['options']['B']}\nC) {sample['options']['C']}\nD) {sample['options']['D']}\n\nReason and output final answer in <ANSWER> tags."}
        ]
        
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        # تولید خروجی
        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_new_tokens=150, 
                temperature=0.1, 
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        
        generated_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
        
        # استخراج حرف از تگ
        match = re.search(r'<ANSWER>\s*([A-D](?:\s*,\s*[A-D])*)\s*</ANSWER>', generated_text)
        if match:
            clean_answer = match.group(1).replace(' ', '')
        else:
            fallback = re.search(r'[A-D](?:\s*,\s*[A-D])*', generated_text)
            clean_answer = fallback.group(0).replace(' ', '') if fallback else "A" # مقدار پیش‌فرض در صورت خطای مدل
            
        # فرمت استاندارد سایت Codabench
        result = {
            "id": sample["id"],
            "prediction": clean_answer
        }
        out_file.write(json.dumps(result) + '\n')

print(f"\n🎯 عملیات تمام شد! فایل {PREDICTIONS_OUTPUT} برای آپلود در Codabench آماده است.")

در حال بارگذاری مدل برای Inference...


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

در حال بارگذاری داده‌های آزمون...


FileNotFoundError: [Errno 2] No such file or directory: 'semeval2026-task12-dataset/dev_data/docs.json'